In [ ]:
# Importações
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.utils import shuffle
import csv

# Caso o NLTK não esteja configurado:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
# Importa os dados extraídos
df_true = pd.read_csv('dados/df_combinedtrue.csv')
df_false = pd.read_csv('dados/df_combinedfake.csv')

In [ ]:
# Carrega o FakeRecogna
df_fakerecogna = pd.read_excel('dados/FakeRecogna.xlsx')

# Filtra apenas as notícias de saúde
df_fakerecogna = df_fakerecogna.loc[df_fakerecogna["Categoria"] == 'saúde']

df_fakerecogna = df_fakerecogna.drop(['Categoria', 'Subtitulo', 'Autor'], axis=1)
df_fakerecogna = df_fakerecogna[['Titulo', 'URL', 'Data', 'Classe', 'Noticia']]
df_fakerecogna = df_fakerecogna.rename(columns = {'Titulo':'title', 'URL': 'link', 'Data': 'date', 'Classe':'classe','Noticia':'corpo_texto'})
df_fakerecogna['classe'] = df_fakerecogna['classe'].astype('int64')

display(df_fakerecogna)

In [ ]:
# Palavras a serem buscadas
keywords = ["vacina", "vacinação", "vacinacao", "covid", "pandemia", "corona", "coronavírus", "coronavirus", "quarentena", "sars-cov-2"]

In [ ]:

# filtra o dataset com o mesmo critério que o de extração
df_fakerecogna = df_fakerecogna[df_fakerecogna['title'].str.contains('|'.join(keywords), case=False, na=False)]

display(df_fakerecogna)

In [ ]:
# Combinar os DataFrames
df_final = pd.concat([df_true, df_false, df_fakerecogna], ignore_index=True)

display(df_final)

In [ ]:
# ----------------------------- PRÉ PROCESSAMENTO ---------------------------- #
# Garante que há conteúdo na coluna de título, corpo do texto e classe
df_final = df_final.dropna(subset=['title', 'corpo_texto', 'classe'])

# Garante que não há notícias repetidas
df_final = df_final.drop_duplicates()
df_final = df_final.reset_index(drop=True)

# Remove maiúsculas
df_final['corpo_texto'] = df_final['corpo_texto'].str.lower()
df_final['title'] = df_final['title'].str.lower()

# Tokeniza
df_final['corpo_texto'] = df_final['corpo_texto'].apply(lambda x: nltk.word_tokenize(x, language='portuguese'))
df_final['title'] = df_final['title'].apply(lambda x: nltk.word_tokenize(x, language='portuguese'))

# Remove stop_words e pontuação
stop_words = set(stopwords.words('portuguese'))

def clean_text(tokens):
    return [re.sub(r'[^\w\s]', '', word) for word in tokens if word not in stop_words and re.sub(r'[^\w\s]', '', word)]

df_final['corpo_texto'] = df_final['corpo_texto'].apply(clean_text)
df_final['title'] = df_final['title'].apply(clean_text)

# Embaralhar e resetar o índice
df_final = shuffle(df_final).reset_index(drop=True)

display(df_final)


In [ ]:
# Salvar o df final
csv_file_path = "dados/df_final.csv"

compression_options = dict(method='zip', archive_name=csv_file_path)
df_final.to_csv(f'dados/df_final.zip', index=False, encoding='utf-8', quoting=csv.QUOTE_ALL, escapechar='\\', compression=compression_options)